In [1]:
import mlflow
import mlflow.sklearn
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd


c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
# ===============================
# Step 1: Load dataset
# ===============================
df = pd.read_csv(r"C:\Users\Deepu\OneDrive\Desktop\Capstone1\reddit_preprocessing.csv")  # Replace with your dataset path
# Columns assumed: 'clean_comment' (text) and 'category' (target)


In [4]:
#  ===============================
# Step 2: Vectorization (TF-IDF)
# ===============================
ngram_range = (1, 3)   # Trigrams
max_features = 1000    # Limit vocab size

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

In [5]:
# ===============================
# Step 3: Apply SMOTE
# ===============================
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# ===============================
# Step 4: Train-test split
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

In [6]:
# ===============================
# Step 5: Optuna Objective Function
# ===============================
def objective_nb(trial):
    params = {
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    }
    model = MultinomialNB(**params)
    model.fit(X_train, y_train)
    return accuracy_score(y_test, model.predict(X_test))



In [7]:
# ===============================
# Step 6: Hyperparameter Tuning
# ===============================
mlflow.set_experiment("ML Algos with HPT")

study = optuna.create_study(direction="maximize")
study.optimize(objective_nb, n_trials=30)
best_params = study.best_params

2025/08/14 15:25:19 INFO mlflow.tracking.fluent: Experiment with name 'ML Algos with HPT' does not exist. Creating a new experiment.
[I 2025-08-14 15:25:20,022] A new study created in memory with name: no-name-db284274-3293-4c9d-a8a7-3be4dee4bc59
[I 2025-08-14 15:25:20,056] Trial 0 finished with value: 0.6760012680967981 and parameters: {'alpha': 0.05869149298175801}. Best is trial 0 with value: 0.6760012680967981.
[I 2025-08-14 15:25:20,183] Trial 1 finished with value: 0.6762126175631407 and parameters: {'alpha': 0.003921517148225753}. Best is trial 1 with value: 0.6762126175631407.
[I 2025-08-14 15:25:20,236] Trial 2 finished with value: 0.6751558702314276 and parameters: {'alpha': 0.2847093325000857}. Best is trial 1 with value: 0.6762126175631407.
[I 2025-08-14 15:25:20,318] Trial 3 finished with value: 0.6762126175631407 and parameters: {'alpha': 0.10215395501812495}. Best is trial 1 with value: 0.6762126175631407.
[I 2025-08-14 15:25:20,416] Trial 4 finished with value: 0.671457

In [8]:
# ===============================
# Step 7: Train Best Model + Log to MLflow
# ===============================
with mlflow.start_run(run_name="NaiveBayes_with_SMOTE_TFIDF_Trigrams"):
    # Log vectorizer settings
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("max_features", max_features)
    mlflow.log_param("imbalance_method", "SMOTE")

    # Log hyperparameters
    for param, value in best_params.items():
        mlflow.log_param(param, value)

    # Train model
    best_model = MultinomialNB(**best_params)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    # Log metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Save & log model
    model_info = mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="best_naivebayes_model",
        registered_model_name="NaiveBayes_TFIDF_Trigrams_SMOTE"
    )

    print("Best Parameters:", best_params)
    print("Model Saved at:", model_info.model_uri)

Best Parameters: {'alpha': 0.002802631141018244}
Model Saved at: runs:/da09880641544ff29edcfc3e226c4323/best_naivebayes_model


Successfully registered model 'NaiveBayes_TFIDF_Trigrams_SMOTE'.
Created version '1' of model 'NaiveBayes_TFIDF_Trigrams_SMOTE'.
